
# Data acquisition controllers
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents

* Writing counter/timer controller - Network Trafic Counter
* Playing with Blender Detector - 2D experimental channel controller

### Network Traffic Counter

* Open the code: `kwrite sardana-training/controllers/templates/NetworkTrafficCTCtrl.py`

### Network Traffic Counter
* Network traffic counter counts bytes that pass via a network interface (both ways)
* Demo:
 * Directly from the system: `cat /proc/net/dev | grep eth0`
 * Using python function:
   * `cd ~/sardana-training/controllers/`
   * `python3 -c "from NetworkTrafficCTCtrl import read_network_counts; print(read_network_counts('eth0'))"`

### [Writing constructor](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#constructor)
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured
* [Define controller property](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#extra-controller-properties) `interface`

### [Writing constructor](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#constructor)
* Counter will work on the following principle:
 * Load of the timer will store the integration time in the cache
 * Start of the counting will latch the current number of bytes
 * Start of the counting will define the end acquisition time (current time + integration time)
* In the constructor we need to instantiate the necessary variables:
 * acq_time = 1
 * acq_end_time = time.time()
 * start_counts = 0

### Instantiate controller
* Deploy controller: `ln -s ~/sardana-training/controllers/templates/NetworkTrafficCTCtrl.py ~/controllers`
* Load it in the system - in spock: `addctrllib NetworkTrafficCTCtrl`
* Create an instance of the controller: `defctrl NetworkTrafficCounterTimerController netctrl interface eth0`
* Ask for controllers state: `netctrl.state()`

### [Implement StateOne](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#get-axis-state)
* Assume state is On
* Only if current time is less then the acq_end_time return Moving state
* IMPORTANT: due to [sardana-org/sardana#621](https://github.com/sardana-org/sardana/issues/621) return an empty status `""`

### [Implement LoadOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#load-a-counter)
 * receives either integration time or monitor counts (negative number) as arguments \*
 * is called only on the controller's timer or monitor
 
 * store the integration time in the cache `acq_time`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### [Implement ReadOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#get-counter-value)
 * returns a single counter value
 * is called multiple times during the acquisition operation

 * latch network bytes
 * return a difference between the current network bytes and `start_counts`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### Instantiate net counter
* Reload controller code: `relctrlcls NetworkTrafficCounterTimerController` 
* Create motor instance: `defelem net netctrl 1`
* Ask for motor state: `net.state()`
* Ask for counter value: `net.value`

### [Implement StartOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#start-a-counter)
 * receives intergation timer or monitor count as argument
 * latch the current bytes to `start_counts` and calcultate the end acquisition time in `acq_end_time`

### Demonstrate single acquisition with net counter
* Reload controller code: `relctrlcls NetworkTrafficCounterTimerController` 
* `ct 1 net` -> zero counts!
* Install `wget` tool and get sardana CHANGELOG while long counting e.g. `ct net 10` execute:
 * `wget https://github.com/sardana-org/sardana/blob/develop/CHANGELOG.md`
